# Profiling Generating Conversion Code for In-memory Representations of Images using a Knowledge Graph of Data Types


In [1]:
import os
import sys

# Assume the current working directory is where the this notebook file is located 
project_dir = os.path.dirname(os.getcwd())
if project_dir not in sys.path:
    sys.path.append(os.path.join(project_dir, "src"))
# Alternatively you can install package directly from the Pypi. 
# %pip install imgdataconvertcodegen

In [2]:
import cProfile
import pstats

from imgdataconvertcodegen import get_conversion_code, _code_generator

def clear_cache_in_code_generator():
    _code_generator._cache = {}

2024-04-08 21:02:13.865151: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 21:02:16.868096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(_code_generator.knowledge_graph)

Knowledge Graph with 553 nodes and 6472 edges.


In [5]:
import cProfile
import pstats

clear_cache_in_code_generator()

with cProfile.Profile() as profile:
    source_image_desc = {"lib": "numpy"}
    target_image_desc = {"lib": "torch", "image_dtype": 'uint8'}
    code = get_conversion_code("source_image", source_image_desc, "target_image", target_image_desc)
    
    
    results = pstats.Stats(profile)
    results.sort_stats(pstats.SortKey.TIME)
    results.print_stats(2)

    print(f'Converson code is \n{code}')

         3028 function calls (2998 primitive calls) in 0.002 seconds

   Ordered by: internal time
   List reduced from 105 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 C:\Users\fech01-admin\anaconda3\envs\kg4idr\Lib\site-packages\networkx\algorithms\shortest_paths\astar.py:12(astar_path)
      694    0.000    0.000    0.000    0.000 C:\Users\fech01-admin\anaconda3\envs\kg4idr\Lib\site-packages\networkx\algorithms\shortest_paths\weighted.py:78(<lambda>)


Converson code is 
import torch
var_e9d7b6a25bd348ada1823b1778d9350d = torch.from_numpy(source_image)
var_ad4cfcfd8dac4760b0fc5af9ac06fc1e = var_e9d7b6a25bd348ada1823b1778d9350d.permute(2, 0, 1)
target_image = var_ad4cfcfd8dac4760b0fc5af9ac06fc1e.unsqueeze(0)


In [7]:
import numpy as np
import torch

clear_cache_in_code_generator()
with cProfile.Profile() as profile:
    source_image = np.random.randint(0, 256, (20, 20, 3), dtype=np.uint8)
    source_image_desc = {"lib": "numpy"}
    target_image_desc = {"lib": "torch", "image_dtype": 'uint8'}
    code = get_conversion_code("source_image", source_image_desc, "actual_image", target_image_desc)
    exec(code, globals())

    results = pstats.Stats(profile)
    results.sort_stats(pstats.SortKey.TIME)
    results.print_stats(2)

    print(f'Converson code is \n{code}')
    expected_image = torch.from_numpy(source_image).unsqueeze(0).permute(0, 3, 1, 2)
    assert torch.equal(actual_image, expected_image), "The actual image does not match the expected image."
    print("\nThe actual image matches the expected image.")

         2289 function calls (2288 primitive calls) in 0.002 seconds

   Ordered by: internal time
   List reduced from 75 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 C:\Users\fech01-admin\anaconda3\envs\kg4idr\Lib\site-packages\networkx\algorithms\shortest_paths\astar.py:12(astar_path)
      694    0.000    0.000    0.000    0.000 C:\Users\fech01-admin\anaconda3\envs\kg4idr\Lib\site-packages\networkx\algorithms\shortest_paths\weighted.py:78(<lambda>)


Converson code is 
import torch
var_eedcd572407048478bbff65713475ed5 = torch.from_numpy(source_image)
var_ba0abb1f0860438880feae1968ee8e3b = var_eedcd572407048478bbff65713475ed5.permute(2, 0, 1)
actual_image = var_ba0abb1f0860438880feae1968ee8e3b.unsqueeze(0)

The actual image matches the expected image.
